In [4]:
import os
import sys
import time
import serial
import math
import numpy as np

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
sys.path.insert(0, os.path.join(os.getcwd(), 'joachim'))
from Thymio import Thymio
#ls /dev/ttyACM*
#sudo chmod 666 /dev/ttyACM0

In [5]:
class robot():
    def __init__(self):
        self.pos_th = [0,0,0]
        self.th = Thymio.serial(port="/dev/ttyACM4", refreshing_rate=0.1) #/dev/ttyACM0 for linux
    def GetPosition(self):
        return self.pos_th
    def measurement_model(self):
        #distance
        x_d = np.linspace(0.5, 5, 10)
        y_d = [16,32.5,46.5,61,79.5,95,109,127.5,143,159.5]
        
        diff_y_d = np.diff(y_d) # Calculating difference list 
        mean_d = np.mean(diff_y_d) #15.94444
        sigma_d = np.std(diff_y_d) #6.877318
        distance_p0 = [1, mean_d, sigma_d]

        #angle
        x_a = np.linspace(0, 9, 10)
        y_a = [0,42,78,114,150,194,231,270,310,347]

        diff_y_a = np.diff(y_a) # Calculating difference list 
        mean_a = np.mean(diff_y_a) 
        sigma_a = np.std(diff_y_a) 
        angle_p0 = [1, mean_a, sigma_a]
        return distance_p0, angle_p0

In [6]:
def Main():
    
    th_m = robot()
    #pos_th = th_m.getPosition()

    distance_p0, angle_p0 = th_m.measurement_model()
    print("distance measurement model: {}\n".format(distance_p0))
    print("angle measurement model: {}".format(angle_p0))
    
if __name__=="__main__": 
    Main() 

distance measurement model: [1, 15.944444444444445, 1.6235154662447173]

angle measurement model: [1, 38.55555555555556, 2.753224820747526]
